In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # Add root dir to path for src/

import numpy as np
import pandas as pd
from src.galactic_dynamics import v_rotation_UDT  # Now imports correctly

# For MassModels_Lelli2016c.mrt
colspecs_mass = [(0, 11), (12, 18), (19, 25), (26, 32), (33, 38), (39, 45), (46, 52), (53, 59), (60, 67), (68, 76)]
names_mass = ['Galaxy', 'D', 'R', 'Vobs', 'e_Vobs', 'Vgas', 'Vdisk', 'Vbul', 'SBdisk', 'SBbul']

mass_models = pd.read_fwf('../data/sparc_database/MassModels_Lelli2016c.mrt', colspecs=colspecs_mass, names=names_mass, skiprows=19, comment='#')

# Convert to numeric
for col in ['R', 'Vobs', 'e_Vobs', 'Vgas', 'Vdisk', 'Vbul', 'SBdisk', 'SBbul']:
    mass_models[col] = pd.to_numeric(mass_models[col], errors='coerce').fillna(0)

# Calculate M_star from SBdisk and SBbul (M/L disk=0.5, bulge=0.7)
mass_models.sort_values(['Galaxy', 'R'], inplace=True)
mass_models['Delta_r'] = mass_models.groupby('Galaxy')['R'].diff().fillna(mass_models['R'])
mass_models['M_disk'] = mass_models['SBdisk'] * 2 * np.pi * mass_models['R'] * mass_models['Delta_r'] * 1e6 * 0.5  # area in pc^2
mass_models['M_bul'] = mass_models['SBbul'] * 2 * np.pi * mass_models['R'] * mass_models['Delta_r'] * 1e6 * 0.7
mass_models['M_point'] = mass_models['M_disk'] + mass_models['M_bul']
mass_models_grouped = mass_models.groupby('Galaxy')['M_point'].sum().reset_index().rename(columns={'M_point': 'Mstar'})

# Loop
galaxies = mass_models['Galaxy'].unique()
chi2_total = 0
success_count = 0
results = []

for gal in galaxies:
    gal_data = mass_models[mass_models['Galaxy'] == gal]
    if gal_data.empty:
        continue
    r = gal_data['R'].values
    v_obs = gal_data['Vobs'].values
    v_err = gal_data['e_Vobs'].values
    M_star = mass_models_grouped[mass_models_grouped['Galaxy'] == gal]['Mstar'].mean()

    # Filter r > 0
    valid = r > 0
    if not np.any(valid):
        continue
    r = r[valid]
    v_obs = v_obs[valid]
    v_err = v_err[valid]
    v_err = np.maximum(v_err, 0.1 * v_obs)  # Paper 10%

    v_pred = v_rotation_UDT(r, M_star)
    
    # Calculate chi2 with try/except for inf/nan
    try:
        diff = (v_obs - v_pred) / v_err
        chi2 = np.sum(diff**2)
        if np.isinf(chi2) or np.isnan(chi2):
            continue
    except:
        continue
    nu = len(r) - 1
    chi2_nu = chi2 / nu if nu > 0 else 0
    print(f"{gal}: χ² = {chi2:.2f}, χ²/ν = {chi2_nu:.2f}")

    chi2_total += chi2
    if chi2_nu < 2.0:
        success_count += 1
    
    results.append({'Galaxy': gal, 'chi2': chi2, 'chi2_nu': chi2_nu})

mean_chi2 = chi2_total / len(results) if len(results) > 0 else 0
total_nu = sum(len(mass_models[mass_models['Galaxy'] == gal]) - 1 for gal in results)
mean_reduced_chi2 = chi2_total / total_nu if total_nu > 0 else 0
success_rate = (success_count / len(results)) * 100 if len(results) > 0 else 0

print(f"Overall χ²: {mean_chi2:.1f}")
print(f"Mean reduced χ²: {mean_reduced_chi2:.1f}")
print(f"Success rate (χ²/ν < 2.0): {success_rate:.0f}%")

# Create results dir if not exists
os.makedirs('../results', exist_ok=True)

# Save summary
pd.DataFrame(results).to_csv('../results/galactic_validation_summary.csv', index=False)

[8.0011619  4.62033878 3.57338859 3.02308802 2.68116309 2.43437826
 2.25957594 2.12348131]
CamB: χ² = 278.31, χ²/ν = 34.79
D512-2: χ² = 180.96, χ²/ν = 60.32
D564-8: χ² = 407.79, χ²/ν = 81.56
D631-7: χ² = 1182.48, χ²/ν = 78.83
DDO064: χ² = 523.22, χ²/ν = 40.25
DDO154: χ² = 997.83, χ²/ν = 90.71
DDO161: χ² = 2091.64, χ²/ν = 69.72
DDO168: χ² = 687.28, χ²/ν = 76.36
DDO170: χ² = 621.62, χ²/ν = 88.80
ESO079-G014: χ² = 698.43, χ²/ν = 49.89
ESO116-G012: χ² = 1037.71, χ²/ν = 74.12
ESO444-G084: χ² = 533.28, χ²/ν = 88.88
ESO563-G021: χ² = 3412.68, χ²/ν = 117.68
F561-1: χ² = 230.94, χ²/ν = 46.19
F563-1: χ² = 804.66, χ²/ν = 50.29
F563-V1: χ² = 78.33, χ²/ν = 15.67
F563-V2: χ² = 297.34, χ²/ν = 33.04
F565-V2: χ² = 411.77, χ²/ν = 68.63
F567-2: χ² = 153.34, χ²/ν = 38.34
F568-1: χ² = 545.86, χ²/ν = 49.62
F568-3: χ² = 740.48, χ²/ν = 43.56
F568-V1: χ² = 625.69, χ²/ν = 44.69
F571-8: χ² = 822.25, χ²/ν = 68.52
F571-V1: χ² = 429.05, χ²/ν = 71.51
F574-1: χ² = 881.70, χ²/ν = 67.82
F574-2: χ² = 32.87, χ²/ν = 8.22
